In [1]:
# Dieser Code ließt die Rohdaten der Stadtwerke aus den Ordner "input" ein, wandelt diese csv. Dateien in ein besseres Format und speichert die neuen csv. Dateien in dem Ornder "output"

import csv
from datetime import datetime, timedelta
import pandas as pd
import os
import itertools


folder = 'input'
files = os.listdir(folder)
#for each file in the input folder
for file in files:
    values = {}
    ID = 0
    with open(f'{folder}/{file}', newline='') as csvfile:
        #"os.path.splitext" ist eine Funktion aus der "os.path"-Bibliothek, die den Dateinamen und die Dateierweiterung aus einem vollständigen Pfad trennt
        filename = os.path.splitext(file)[0]
        #In diesem Fall wird die Dateierweiterung entfernt, indem "[0]" verwendet wird, um das erste Element des Rückgabewerts von "os.path.splitext" (den Dateinamen) zu erhalten
        data = csv.reader(csvfile, delimiter = ";")
        #skip the first 3 lines
        next(data)
        #first occurrence when opening file (initialisation)
        row = next(itertools.islice(data, 1, 2))
        ID = row[10]
        day = datetime.strptime(row[0], '%d.%m.%y')
        date = datetime.strptime(row[0]+" "+row[1], '%d.%m.%y %H:%M:%S')
        date = date + timedelta(hours=-1)

        for value in row[15::2]:
            date = date + timedelta(hours=1)
            values[date] = value
        for row in data: 
            date = datetime.strptime(row[0]+" "+row[1], '%d.%m.%y %H:%M:%S')
            date = date + timedelta(hours=-1)
            #while still same smartmeter ID
            if ID == row[10]:
                for value in row[15::2]:
                    date = date + timedelta(hours=1)
                    values[date] = value

            #start of new smartmeter
            else : 
                #save current output
                dataframe = pd.DataFrame.from_dict(values, orient='index')
                os.makedirs(f'output/{filename}', exist_ok=True)  
                dataframe.to_csv(f'output/{filename}/{ID}.csv')

                ID = row[10]
                for value in row[15::2]:
                    date = date + timedelta(hours=1)
                    values[date] = value
                
